In [7]:
#import cv2
import numpy as np
np.set_printoptions(threshold = np.inf)
#import pydicom
import matplotlib.pyplot as plt
#import dicom2nifti
#import nibabel as nib
import scipy
import sys
#from sklearn import preprocessing
from skimage import exposure
import os
import pandas as pd
from scipy.ndimage import label
from scipy.ndimage.morphology import generate_binary_structure
import SimpleITK as sitk
print(sitk.Version())
from pathlib import Path
#from nipype.interfaces.ants import N4BiasFieldCorrection
import skimage.io as io
#import fsl
import itk
from shutil import copy

SimpleITK Version: 2.3.1 (ITK 5.3)
Compiled: Nov  1 2023 12:24:28



Please use `generate_binary_structure` from the `scipy.ndimage` namespace, the `scipy.ndimage.morphology` namespace is deprecated.


## Function 

1. CT_prerprocess: CT preprocessing code
2. ReOri_ReDir: reset the origin and direction to fixed
3. show_img: show the image by dimension
4. load_img: load data with nifti/array foramt
5. coRegis: mapping the moving img to the fixed img center

In [11]:
def adjust_array(ori_direction, ori_array):
    print('ori_dir, ori_origin: ',ori_array.GetDirection(), ', ', ori_array.GetOrigin())
    ori_ori, ori_dir = ori_array.GetOrigin(),ori_array.GetDirection()
    ori_array = sitk.GetArrayFromImage(ori_array)
    print(ori_array.shape)
    if ori_direction[0] == -1.0:
        print('flip on axis 2')
        np.flip(ori_array, axis = 2)
    if ori_direction[4] == -1.0:
        np.flip(ori_array, axis = 1)
        print('flip on axis 1')
    if ori_direction[8] == -1.0:
        np.flip(ori_array, axis = 0)
        print('flip on axis 0')
        
    transfer_array = sitk.GetImageFromArray(ori_array)
    transfer_array.SetOrigin(ori_ori)
    print('transfer_dir, transfer_origin: ',transfer_array.GetDirection(), ', ', transfer_array.GetOrigin())   
    
    return transfer_array

In [12]:
def adjust_direction(object_example, ori_example):
    print('ori_dir, ori_origin: ',ori_example.GetDirection(), ', ', ori_example.GetOrigin())
    ori_ori = ori_example.GetOrigin()
    ori_dir = ori_example.GetDirection()
    print('object_dir, object_ori: ',object_example.GetDirection(), ', ', object_example.GetOrigin())
    object_ori= object_example.GetOrigin()
    object_dir = object_example.GetDirection()
#     ori_example = sitk.GetArrayFromImage(ori_example)
    ori_example = sitk.GetArrayFromImage(ori_example)
    print(ori_example.shape)
    if ori_dir[0] != object_dir[0]:
        print('different axis 2')
        np.flip(ori_example, axis = 2)
        
    if ori_dir[4] != object_dir[4]:
        print('different axis 1')
        np.flip(ori_example, axis = 1)
        print(ori_example.shape)
        
    if ori_dir[8] != object_dir[8]:
        print('different axis 0')
        np.flip(ori_example, axis = 0)
    
    print(ori_example.shape)
    transfer_array = sitk.GetImageFromArray(ori_example)
#     print(ori_example_output.shape)
    transfer_array = transfer_array.SetOrigin(object_ori)
 #   print(ori_example_output.shape)
#     print('transfer_dir, transfer_origin: ',ori_example_output.GetDirection(), ', ', ori_example_output.GetOrigin())   
    
    return transfer_array

In [ ]:
def CT_preprocess_newest(nif_dat_path, new_spacing ,level, window, scale, fltsize, Origin):
    """ Given an 3D nifti data, scale into window and level. Scale between
    expects a tuple (new_min, new_max) that determines the new range.The preprocess 
    also do the resampling and registrationof the CT image.
    Works with both 2D and 3D data.
    
    nif_dat: input the data from sitkImage Read
    new_spacing: the voxel size: [voxel[0], voxel[1], voxel[2]]
    level : center of window, window = width of window.
    scale : (min_scale, max_scale) to rescale in-window values to.
    filt_size : the median filter kernal size; Ex:fltsize = 5  = median_filter(5,5,5)
    Origin: The nifti data origin

    """
    
    MNI_152 = sitk.ReadImage(r"S:\StrokeAI\Try\mni_icbm152_nlin_asym_09a\mni_icbm152_t1_tal_nlin_asym_09a.nii") 
    nif_dat = sitk.ReadImage(nif_dat_path) 

    fixed_img = sitk.Cast(MNI_152, sitk.sitkFloat32)
    moving_img = sitk.Cast(nif_dat, sitk.sitkFloat32)
    
    initial_transform = sitk.CenteredTransformInitializer(
        fixed_img, # Note these are purposefuly reversed!
        moving_img,# Note these are purposefuly reversed!
        sitk.Euler3DTransform(),
        sitk.CenteredTransformInitializerFilter.GEOMETRY)
    moving_resampled = sitk.Resample(
        moving_img,       # Note these are purposefuly reversed!
        fixed_img,      # Note these are purposefuly reversed!
        initial_transform,
        sitk.sitkLinear,  # TODO: use different interpolator?
        0.0,              # Note(Jacob): default value
        moving_img.GetPixelID(),
    )
    npmoving = sitk.GetArrayFromImage(moving_resampled)
    #plt.figure(1)
    #plt.imshow(npmoving[80,:,:],cmap ='gray')
    #plt.colorbar()
    itk_fix = sitk2itk(fixed_img,3)
    itk_mov = sitk2itk(moving_resampled,3)
    #print(itk.size(itk_mov))
    parameter_object = itk.ParameterObject.New()
    parameter_map_rigid = parameter_object.GetDefaultParameterMap('rigid')
    parameter_object.AddParameterMap(parameter_map_rigid)
    result_img, result_transform_parameters = itk.elastix_registration_method(itk_fix,itk_mov, parameter_object = parameter_object,log_to_console = True)
    result_img = itk2sitk(result_img)
    #print(result_img.GetSize())
    #print(result_img.GetSize())
    npmoving = sitk.GetArrayFromImage(result_img)
    #plt.figure(2)
    #plt.imshow(npmoving[80,:,:],cmap ='gray')
    #plt.colorbar()
    #print(npmoving[80,:,:].shape)
    # denoise the image by 3D median filter
    npmoving = scipy.ndimage.median_filter(npmoving, size = (fltsize,fltsize,fltsize))    
    # Extract the brain
    im_size = npmoving.shape
    #print(im_size)
    upper = level + window/2.0
    lower = level - window/2.0
    npmoving[npmoving < 0.0] = 0.0
    #npmoving[npmoving > 400.0] = 0.0
    npmoving[npmoving > upper] = upper
    npmoving[npmoving < lower] = lower
    #plt.figure(3)
    #plt.imshow(npmoving[80,:,:],cmap ='gray')
    #plt.colorbar()
    
    re_img = np.zeros(im_size)
#    upper = level + window/2.0
#   lower = level - window/2.0
    for num in range(im_size[0]):
        img1 = npmoving[num, :, :]
        new_img = img1.copy()        
        new_img[new_img > upper] = upper
        new_img[new_img < lower] = lower

    # Delete the Gantry
        if new_img.max() > 0: 
            if np.max(img1)<=1e-10:
                low_t=0
            else:
                low_t=1e-10
            ref, bi_img = cv2.threshold(img1 ,low_t,1,cv2.THRESH_BINARY)
            bi_img = np.uint8(bi_img)
            num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(bi_img)
            op = np.zeros(new_img.shape)
            pixel_label = np.resize(labels, labels.shape[0]*labels.shape[1])            
            num1 = np.delete(np.bincount(pixel_label),0)
            label_brain = np.where(num1 == num1.max())[0]+1
            mask = labels == label_brain
            brain_pos = np.where(mask[:,:] == True)
            op[mask] = new_img[brain_pos]
            op.astype(float)
            re_img[num, :, :] = op
        elif new_img.max() == 0:
            op = new_img
            re_img[num, :, :] = op
    re_img = (re_img -lower)/ (upper-lower) * (scale[1] - scale[0])
    re_img[re_img == 1] = 0
    re_img = sitk.GetImageFromArray(re_img)

    print('re_img size: ', re_img.GetSize())
    print('re_img spacing: ', re_img.GetSpacing())
   
    return re_img

In [14]:
def CT_preprocess_final(nif_dat_path, new_spacing ,level, window, scale, fltsize, Origin):
    """ Given an 3D nifti data, scale into window and level. Scale between
    expects a tuple (new_min, new_max) that determines the new range.The preprocess 
    also do the resampling and registrationof the CT image.
    Works with both 2D and 3D data.
    
    nif_dat: input the data from sitkImage Read
    new_spacing: the voxel size: [voxel[0], voxel[1], voxel[2]]
    level : center of window, window = width of window.
    scale : (min_scale, max_scale) to rescale in-window values to.
    filt_size : the median filter kernal size; Ex:fltsize = 5  = median_filter(5,5,5)
    Origin: The nifti data origin

    """
    
    MNI_152 = sitk.ReadImage(r"S:\StrokeAI\Try\mni_icbm152_nlin_asym_09a\mni_icbm152_t1_tal_nlin_asym_09a.nii") 
    nif_dat = sitk.ReadImage(nif_dat_path) 

    fixed_img = sitk.Cast(MNI_152, sitk.sitkFloat32)
    moving_img = sitk.Cast(nif_dat, sitk.sitkFloat32)
    
    initial_transform = sitk.CenteredTransformInitializer(
        fixed_img, # Note these are purposefuly reversed!
        moving_img,# Note these are purposefuly reversed!
        sitk.Euler3DTransform(),
        sitk.CenteredTransformInitializerFilter.GEOMETRY)
    moving_resampled = sitk.Resample(
        moving_img,       # Note these are purposefuly reversed!
        fixed_img,      # Note these are purposefuly reversed!
        initial_transform,
        sitk.sitkLinear,  # TODO: use different interpolator?
        0.0,              # Note(Jacob): default value
        moving_img.GetPixelID(),
    )
    npmoving = sitk.GetArrayFromImage(moving_resampled)
    #plt.figure(1)
    #plt.imshow(npmoving[80,:,:],cmap ='gray')
    #plt.colorbar()
    itk_fix = sitk2itk(fixed_img,3)
    itk_mov = sitk2itk(moving_resampled,3)
    #print(itk.size(itk_mov))
    parameter_object = itk.ParameterObject.New()
    parameter_map_rigid = parameter_object.GetDefaultParameterMap('rigid')
    parameter_object.AddParameterMap(parameter_map_rigid)
    result_img, result_transform_parameters = itk.elastix_registration_method(itk_fix,itk_mov, parameter_object = parameter_object,log_to_console = True)
    result_img = itk2sitk(result_img)
    #print(result_img.GetSize())
    #print(result_img.GetSize())
    npmoving = sitk.GetArrayFromImage(result_img)
    #plt.figure(2)
    #plt.imshow(npmoving[80,:,:],cmap ='gray')
    #plt.colorbar()
    #print(npmoving[80,:,:].shape)
    # denoise the image by 3D median filter
    npmoving = scipy.ndimage.median_filter(npmoving, size = (fltsize,fltsize,fltsize))    
    # Extract the brain
    im_size = npmoving.shape
    #print(im_size)
    upper = level + window/2.0
    lower = level - window/2.0
    npmoving[npmoving < 0.0] = 0.0
    npmoving[npmoving > 300] = 0.0
    npmoving[npmoving > upper] = upper
    npmoving[npmoving < lower] = lower
    #plt.figure(3)
    #plt.imshow(npmoving[80,:,:],cmap ='gray')
    #plt.colorbar()
    
    re_img = np.zeros(im_size)
#    upper = level + window/2.0
#   lower = level - window/2.0
    for num in range(im_size[0]):
        img1 = npmoving[num, :, :]
        new_img = img1.copy()        
        new_img[new_img > upper] = upper
        new_img[new_img < lower] = lower

    # Delete the Gantry
        if new_img.max() > 0: 
            if np.max(img1)<=1e-10:
                low_t=0
            else:
                low_t=1e-10
            ref, bi_img = cv2.threshold(img1 ,low_t,1,cv2.THRESH_BINARY)
            bi_img = np.uint8(bi_img)
            num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(bi_img)
            op = np.zeros(new_img.shape)
            pixel_label = np.resize(labels, labels.shape[0]*labels.shape[1])            
            num1 = np.delete(np.bincount(pixel_label),0)
            label_brain = np.where(num1 == num1.max())[0]+1
            mask = labels == label_brain
            brain_pos = np.where(mask[:,:] == True)
            op[mask] = new_img[brain_pos]
            op.astype(float)
            re_img[num, :, :] = op
        elif new_img.max() == 0:
            op = new_img
            re_img[num, :, :] = op
    re_img = (re_img -lower)/ (upper-lower) * (scale[1] - scale[0])
    re_img = sitk.GetImageFromArray(re_img)

    print('re_img size: ', re_img.GetSize())
    print('re_img spacing: ', re_img.GetSpacing())
   
    return re_img

In [15]:
def CT_preprocess_skulloff(input_nif ,level, window, scale, fltsize):
    """ Given an 3D nifti data, scale into window and level. Scale between
    expects a tuple (new_min, new_max) that determines the new range.The preprocess 
    also do the resampling and registrationof the CT image.
    Works with both 2D and 3D data.
    
    nif_dat: input the data from sitkImage Read
    new_spacing: the voxel size: [voxel[0], voxel[1], voxel[2]]
    level : center of window, window = width of window.
    scale : (min_scale, max_scale) to rescale in-window values to.
    filt_size : the median filter kernal size; Ex:fltsize = 5  = median_filter(5,5,5)
    Origin: The nifti data origin

    """
    input_ori, input_dir = input_nif.GetOrigin(),input_nif.GetDirection()
    print('input_origin: ',input_ori)
    npmoving = sitk.GetArrayFromImage(input_nif)
    # denoise the image by 3D median filter
    #npmoving = scipy.ndimage.median_filter(npmoving, size = (fltsize,fltsize,fltsize))    
    # Extract the brain
    im_size = npmoving.shape
    #print(im_size)
    upper = level + window/2.0
    lower = level - window/2.0
    npmoving[npmoving < 0.0] = 0.0
#     npmoving[npmoving > 400.0] = 0.0
    #npmoving[npmoving > upper] = upper
   # npmoving[npmoving < lower] = lower
    re_img = np.zeros(im_size)
    for num in range(im_size[0]):
        img1 = npmoving[num, :, :]
        new_img = img1.copy()        
        new_img[new_img > upper] = upper
        new_img[new_img < lower] = lower
        op = new_img
        re_img[num, :, :] = op

     # Delete the Gantry
 #       if new_img.max() > 0: 
      #      if np.max(img1)<=1e-10:
    #            low_t=0
       #     else:
     #           low_t=1e-10
     #    #   ref, bi_img = cv2.threshold(img1 ,low_t,1,cv2.THRESH_BINARY)
       #     bi_img = np.uint8(bi_img)
#             num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(bi_img)
#             op = np.zeros(new_img.shape)
#             pixel_label = np.resize(labels, labels.shape[0]*labels.shape[1])            
#             num1 = np.delete(np.bincount(pixel_label),0)
#             label_brain = np.where(num1 == num1.max())[0]+1
#             mask = labels == label_brain
#             brain_pos = np.where(mask[:,:] == True)
#             op[mask] = new_img[brain_pos]
#             op.astype(float)
#             re_img[num, :, :] = op
#         elif new_img.max() == 0:

    re_img = (re_img -lower)/ (upper-lower) * (scale[1] - scale[0])
#    re_img[re_img == 1] = 0
    re_img = sitk.GetImageFromArray(re_img)
    re_img.SetOrigin(input_ori)
#     re_img.SetDirection(input_dir)
    print('re_img size: ', re_img.GetSize())
    print('re_img spacing: ', re_img.GetSpacing())
   # print('re_img type: ', re_img.type())
   
    return re_img

In [16]:
def ReOri_ReDir(input_data,target_data, Origin):
    input_data.SetOrigin(Origin)
    dcn = target_data.GetDirection()
    dcn = np.asarray(dcn)
    dcn1 = np.round_(dcn)
    dcn1 = tuple(dcn1)
    input_data.SetDirection(dcn1)
    print(input_data.GetOrigin())
    print(input_data.GetDirection())
    return input_data

In [17]:
def show_img(data,label,d, mode,alpha_num):
    """
    data: input image data
    label: input label data
    d: which dimension to print
    mode:show single image or two images ["single"], ["label"]
    alpha: control image's transparency
    
    """
    if mode == "single":
        if d == 0:
            for i in range(data.shape[0]):
                io.imshow(data[i,:,:],cmap ='gray')
                print(i)
                io.show()
        elif d == 1:
            for i in range(data.shape[0]):
                io.imshow(data[:,i,:],cmap ='gray')
                print(i)
                io.show()
        elif d == 2:
            for i in range(data.shape[0]):
                io.imshow(data[:,:,i],cmap ='gray')
                print(i)
                io.show()
    if mode == "label":
        if d == 0:
            for i in range(data.shape[0]):
                io.imshow(data[i,:,:],cmap ='gray',alpha = alpha_num)
                io.imshow(label[i,:,:],cmap = 'Greens',alpha = 1.0-alpha_num)
                print(i)
                io.show()
        elif d == 1:
            for i in range(data.shape[0]):
                io.imshow(data[0,i,:],cmap ='gray',alpha = alpha_num)
                io.imshow(label[:,i,:],cmap = 'Greens',alpha = 1.0-alpha_num)
                print(i)
                io.show()
        elif d == 2:
            for i in range(data.shape[0]):
                io.imshow(data[:,:,i],cmap ='gray',alpha = alpha_num)
                io.imshow(label[:,:,i],cmap = 'Greens',alpha = 1.0-alpha_num)
                print(i)
                io.show()


In [18]:
def load_img(data):
    nif_dat = sitk.ReadImage(data)
    array_dat = sitk.GetArrayFromImage(nif_dat)
    return nif_dat,array_dat

In [19]:
def sitk2itk(sitk_img, IMAGE_DIMENSION):
    npsitk_img = sitk.GetArrayFromImage(sitk_img)
    itk_img = itk.GetImageFromArray(npsitk_img)
    itk_img.SetOrigin(sitk_img.GetOrigin())
    itk_img.SetSpacing(sitk_img.GetSpacing())
    itk_img.SetDirection(itk.GetMatrixFromArray(np.reshape(np.array(sitk_img.GetDirection()), [IMAGE_DIMENSION]*2)))
    return itk_img
def itk2sitk(itk_img):
    npitk_img = itk.GetArrayFromImage(itk_img)
    sitk_img = sitk.GetImageFromArray(npitk_img, isVector = itk_img.GetNumberOfComponentsPerPixel()>1)
    sitk_img.SetOrigin(tuple(itk_img.GetOrigin()))
    sitk_img.SetSpacing(tuple(itk_img.GetSpacing()))
    sitk_img.SetDirection(itk.GetArrayFromMatrix(itk_img.GetDirection()).flatten())
    return sitk_img

In [20]:
# code resourse: https://www.atyun.com/23342.html
def resample_img(itk_image,out_spacing, is_label=False):
    
    # Resample images to mm spacing with SimpleITK
    original_spacing = itk_image.GetSpacing()
    original_size = itk_image.GetSize()
    
    print('itk_image_spacing ', original_spacing)
    print('itk_image_size ', original_size)
    print('output_spacing ', out_spacing)
    out_size = [
        int(np.round(original_size[0] * (original_spacing[0] / out_spacing[0]))),
        int(np.round(original_size[1] * (original_spacing[1] / out_spacing[1]))),
        int(np.round(original_size[2] * (original_spacing[2] / out_spacing[2])))]
    print('output_size ', out_size)
    resample = sitk.ResampleImageFilter()
    resample.SetOutputSpacing(out_spacing)
    resample.SetSize(out_size)
    resample.SetOutputDirection(itk_image.GetDirection())
    resample.SetOutputOrigin(itk_image.GetOrigin())
    resample.SetTransform(sitk.Transform())
    resample.SetDefaultPixelValue(itk_image.GetPixelIDValue())

    if is_label:
        resample.SetInterpolator(sitk.sitkNearestNeighbor)
    else:
        resample.SetInterpolator(sitk.sitkNearestNeighbor)

    return resample.Execute(itk_image)



In [21]:
def coRegis_final(fixed_img, moving_img, moving_label,metric_method):
    """
    
    This code is help doing the co-registration between CT-image and MR Image/ MRI lesion label
    We do the registration first of the CT image and MR image. After that, we use the transformation
    matrix to transfer MR label to CT-label
    
    fixed_img: CT Image
    moving_img: MR Image
    moving_label: MR label
    metric_method:
    ["extremity MRI"]: intra-subject; affine transformation, mutual information metric
    ["CTMR-based"]:intra-subject; multi-resolution (4)-
    rigid + B-spline transformation, Mutual Information metric (Mattes) with Adaptive Stochastic Gradient Descent optimizer
    
    We tranfer MR image/ MR label into CT space by initial transform and do the co-registration,
    the MR image and MR label spacing/origin will change and same with CT image
    """
    # Generate a centering transform based on the images
    IMAGE_DIMENSION = 3
    fixed_img = CT_preprocess_skulloff(fixed_img,40, 80, (0.0,1.0),1)
    fixed_img = sitk.Cast(fixed_img, sitk.sitkFloat32)
    moving_img = sitk.Cast(moving_img, sitk.sitkFloat32)

    
    #fixed_img = CT_preprocess_skulloff(fixed_img,40, 80, (0.0,1.0),3)
    fixed_img = adjust_direction(moving_img,fixed_img)
    processed_ct  = fixed_img
    print(fixed_img.GetSize())
    fixed_flip = fixed_img
    fixed_Origin = fixed_img.GetOrigin()
    print(fixed_flip.GetSize())
    
    print('fixed_ori_direction:',fixed_flip.GetDirection())

    moving_img = sitk.Cast(moving_img, sitk.sitkFloat32)
    moving_ori = moving_img.GetOrigin()
#     moving_img = sitk.GetArrayFromImage(moving_img)
#     moving_img  = sitk.GetImageFromArray(moving_img)
#     moving_img.SetOrigin(fixed_Origin)
    #moving_img = adjust_array(moving_img.GetDirection(),moving_img)

    moving_flip = moving_img
    print('moving_ori_direction: ',moving_flip.GetDirection())

    label_img = sitk.Cast(moving_label, sitk.sitkFloat32)
    label_ori = label_img.GetOrigin()
#     label_img = sitk.GetArrayFromImage(label_img)
#     label_img  = sitk.GetImageFromArray(label_img)
#     label_img.SetOrigin(fixed_Origin)
    #label_img = adjust_array(label_img.GetDirection(),label_img)
    label_flip = label_img
    print('label_ori_direction: ',label_flip.GetDirection())
    
    initial_transform1 = sitk.CenteredTransformInitializer(
        fixed_flip, # Note these are purposefuly reversed!
        moving_flip,# Note these are purposefuly reversed!
        sitk.Euler3DTransform(),
        sitk.CenteredTransformInitializerFilter.GEOMETRY)
#     initial_transform2 = sitk.CenteredTransformInitializer(
#         fixed_flip, # Note these are purposefuly reversed!
#         label_flip,# Note these are purposefuly reversed!
#         sitk.Euler3DTransform(),
#         sitk.CenteredTransformInitializerFilter.GEOMETRY)
    
    moving_resampled = sitk.Resample(
        moving_flip,       # Note these are purposefuly reversed!
        fixed_flip,      # Note these are purposefuly reversed!
        initial_transform1,
        sitk.sitkNearestNeighbor,# TODO: use different interpolator?
        0.0,              # Note(Jacob): default value
        moving_flip.GetPixelID(),
    )
    label_resampled = sitk.Resample(
        label_flip,       # Note these are purposefuly reversed!
        fixed_flip,      # Note these are purposefuly reversed!
        initial_transform1,
        sitk.sitkNearestNeighbor,# TODO: use different interpolator?
        0.0,              # Note(Jacob): default value
        label_flip.GetPixelID(),
    )
    print('Fixed size ', fixed_flip.GetSize())
    print('Moving size ', moving_flip.GetSize())
    print('Moving Resampled size', moving_resampled.GetSize())
    print('Label Resampled size', label_resampled.GetSize())

    print('Fixed space ', fixed_flip.GetSpacing())
    print('Moving space ', moving_flip.GetSpacing())
    print('Resampled space', moving_resampled.GetSpacing())
    print('Label Resampled space', label_resampled.GetSpacing())
    
    print('Fixed origin ', fixed_flip.GetOrigin())
    print('Moving origin ', moving_flip.GetOrigin())
    print('Resampled origin ', moving_resampled.GetOrigin())
    print('Label Resampled origin', label_resampled.GetOrigin())

    itk_fix = sitk2itk(fixed_flip,3)
    itk_mov = sitk2itk(moving_flip,3)
    itk_label = sitk2itk(label_flip,3)
    parameter_object1 = itk.ParameterObject.New()
    parameter_map_rigid1 = parameter_object1.GetDefaultParameterMap('rigid')
    parameter_map_rigid1['FinalBSplineInterpolationOrder'] = ['0']
    parameter_object1.AddParameterMap(parameter_map_rigid1)
    parameter_map_affine1 = parameter_object1.GetDefaultParameterMap('affine')
    parameter_map_affine1['FinalBSplineInterpolationOrder'] = ['0']
    parameter_object1.AddParameterMap(parameter_map_affine1)
    
    if metric_method == 'extremity MRI':
        parameter_object1.AddParameterFile(r'S:\StrokeAI\Try\co-regis\ElastixSources\Par0040affine.txt')
    if metric_method == 'CTMR-based':
        parameter_object1.AddParameterFile(r'S:\StrokeAI\Try\co-regis\ElastixSources\Par0043rigid.txt')
    if metric_method == 'MRT1':
        parameter_object1.AddParameterFile(r'S:\StrokeAI\Try\co-regis\ElastixSources\Par0002.fs.MI.rigid.RandomCoordinate.txt')
    if metric_method == 'MRHASTE':
        parameter_object1.AddParameterFile(r'S:\StrokeAI\Try\co-regis\ElastixSources\Par0010affine.txt')
        parameter_object1.AddParameterFile(r'S:\StrokeAI\Try\co-regis\ElastixSources\Par0010bspline.txt')

    result_img, result_transform_parameters = itk.elastix_registration_method(itk_fix,itk_mov, parameter_object = parameter_object1,log_to_console = True)
    print('itk_fix origin ',itk_fix.GetOrigin())
    print('itk_fix_direction ', itk_fix.GetDirection())
    print('itk_fix spacing ',itk_fix.GetSpacing())
    print('itk_mov origin ',itk_mov.GetOrigin())
    print('itk_mov spacing ',itk_mov.GetSpacing())
    print('itk_mov_direction ', itk_mov.GetDirection())
    print('result_img origin ',result_img.GetOrigin())
    print('result_img spacing ',result_img.GetSpacing())
    print('result_img_direction ', result_img.GetDirection())
    
    result_img = itk2sitk(result_img)
    print('moving_result_direction: ',result_img.GetDirection())
    #print('result_img_transfer spacing ',result_img.GetSpacing())
    
    itk_label = sitk2itk(label_resampled,3)
    print('Label_origin origin ', itk_label.GetOrigin())
    print('Label_origin space ', itk_label.GetSpacing())
    
    transform_label_img1 =itk.transformix_filter(itk_label, result_transform_parameters)
    
    #result_img2, result_transform_parameters2 = itk.elastix_registration_method(itk_fix,result_img1, parameter_object = parameter_object2)
    #transform_label_img2 =itk.transformix_filter(transform_label_img1,result_transform_parameters2)

    label_result =itk2sitk(transform_label_img1)
    print('label_result_direction: ',label_result.GetDirection())
    #result_img2 = itk2sitk(result_img2)
    ct_img = itk2sitk(itk_fix)
    print('ct_result_direction: ',ct_img.GetDirection())
    print('Label_changed origin ', label_result.GetOrigin())
    print('Label_changed space ', label_result.GetSpacing())
    print('Label_changed size ', label_result.GetSize())
#     print('Label_changed size ', itk_label.GetSize())
    return label_result , result_img,label_resampled ,moving_resampled, result_transform_parameters, moving_flip, label_flip,ct_img

In [22]:
def LPI2RAS(input_img_path,out_path):
    import nibabel as nib
    print(input_img_path)
    print(out_path)
    img = nib.load(input_img_path)
    img = nib.as_closest_canonical(img)
    img_np = img.get_fdata(dtype = np.float32)
    nib.save(img, out_path) 
    return print('LPI2RAS_Finished')


In [23]:
def coRegis_fix(target_fixed, target_moving, target_label):
    """
    
    This code is help doing the co-registration between CT-image and MR Image/ MRI lesion label
    We do the registration first of the CT image and MR image. After that, we use the transformation
    matrix to transfer MR label to CT-label
    
    fixed_img: CT Image
    moving_img: MR Image
    moving_label: MR label
    metric_method:
    ["extremity MRI"]: intra-subject; affine transformation, mutual information metric
    ["CTMR-based"]:intra-subject; multi-resolution (4)-
    rigid + B-spline transformation, Mutual Information metric (Mattes) with Adaptive Stochastic Gradient Descent optimizer
    
    We tranfer MR image/ MR label into CT space by initial transform and do the co-registration,
    the MR image and MR label spacing/origin will change and same with CT image
    """
    

    # Image Prepocessing and Casting
    IMAGE_DIMENSION = 3
    fixed_ori, fixed_dir = target_fixed.GetOrigin(), target_fixed.GetDirection()

    fixed_img = CT_preprocess_skulloff(target_fixed,40, 80, (0.0,1.0),1)
    fixed_img.SetDirection(fixed_dir)
    fixed_img = sitk.Cast(fixed_img, sitk.sitkFloat32)
    print(fixed_img.GetOrigin())
    moving_img = sitk.Cast(target_moving, sitk.sitkFloat32)
    label_img = sitk.Cast(target_label, sitk.sitkFloat32)

    print(label_img.GetDirection())
    print(moving_img.GetDirection())
    print(fixed_img.GetSize())
    print('fixed_ori_direction:',fixed_img.GetDirection())
    print('moving_ori_direction: ',moving_img.GetDirection())
    
    # Initial Transformation
    initial_transform = sitk.CenteredTransformInitializer(
        fixed_img, # Note these are purposefuly reversed!
        moving_img,# Note these are purposefuly reversed!
        sitk.Euler3DTransform(),
        sitk.CenteredTransformInitializerFilter.GEOMETRY)
    
    # Image Resampling
    moving_resampled = sitk.Resample(
        moving_img,       # Note these are purposefuly reversed!
        fixed_img,      # Note these are purposefuly reversed!
        initial_transform,
        sitk.sitkNearestNeighbor,# TODO: use different interpolator?
        0.0,              # Note(Jacob): default value
        moving_img.GetPixelID(),
    )
    label_resampled = sitk.Resample(
        label_img,       # Note these are purposefuly reversed!
        fixed_img,      # Note these are purposefuly reversed!
        initial_transform,
        sitk.sitkNearestNeighbor,  # TODO: use different interpolator?
        0.0,              # Note(Jacob): default value
        label_img.GetPixelID(),
    )
    print('Fixed size ', fixed_img.GetSize())
    print('Moving size ', moving_img.GetSize())
    print('Moving Resampled size', moving_resampled.GetSize())
    print('Label Resampled size', label_resampled.GetSize())

    print('Fixed space ', fixed_img.GetSpacing())
    print('Moving space ', moving_img.GetSpacing())
    print('Resampled space', moving_resampled.GetSpacing())
    print('Label Resampled space', label_resampled.GetSpacing())
    
    print('Fixed origin ', fixed_img.GetOrigin())
    print('Moving origin ', moving_img.GetOrigin())
    print('Moving Resampled origin ', moving_resampled.GetOrigin())
    print('Label Resampled origin', label_resampled.GetOrigin())

    itk_fix = sitk2itk(fixed_img,3)
    itk_mov = sitk2itk(moving_resampled,3)
    itk_label = sitk2itk(label_resampled,3)
    parameter_object1 = itk.ParameterObject.New()
    parameter_map_rigid1 = parameter_object1.GetDefaultParameterMap('rigid')
    parameter_map_rigid1['FinalBSplineInterpolationOrder'] = ['0']
    parameter_object1.AddParameterMap(parameter_map_rigid1)
    parameter_map_affine1 = parameter_object1.GetDefaultParameterMap('affine')
    parameter_map_affine1['FinalBSplineInterpolationOrder'] = ['0']
    parameter_object1.AddParameterMap(parameter_map_affine1)

    result_img, result_transform_parameters = itk.elastix_registration_method(itk_fix,itk_mov, parameter_object = parameter_object1,log_to_console = True)
    print('itk_fix origin ',itk_fix.GetOrigin())
    print('itk_fix_direction ', itk_fix.GetDirection())
    print('itk_fix spacing ',itk_fix.GetSpacing())
    print('itk_mov origin ',itk_mov.GetOrigin())
    print('itk_mov spacing ',itk_mov.GetSpacing())
    print('itk_mov_direction ', itk_mov.GetDirection())
    print('result_img origin ',result_img.GetOrigin())
    print('result_img spacing ',result_img.GetSpacing())
    print('result_img_direction ', result_img.GetDirection())
    
    result_img = itk2sitk(result_img)
    print('moving_result_direction: ', result_img.GetDirection())
    
    itk_label = sitk2itk(label_resampled,3)
    print('Label_origin origin ', itk_label.GetOrigin())
    print('Label_origin space ', itk_label.GetSpacing())
    
    transform_label_img1 =itk.transformix_filter(itk_label, result_transform_parameters)

    label_result =itk2sitk(transform_label_img1)
    print('label_result_direction: ',label_result.GetDirection())
    #result_img2 = itk2sitk(result_img2)
    ct_img = itk2sitk(itk_fix)
    ct_img.SetDirection(fixed_dir)
    print('ct_result_direction: ',ct_img.GetDirection())
    print('Label_changed origin ', label_result.GetOrigin())
    print('Label_changed space ', label_result.GetSpacing())
    print('Label_changed size ', label_result.GetSize())
    
    return label_result , result_img, result_transform_parameters,ct_img

In [24]:
def coRegis_skulloff(fixed_img, moving_img, moving_label):#,metric_method):
    """
    
    This code is help doing the co-registration between CT-image and MR Image/ MRI lesion label
    We do the registration first of the CT image and MR image. After that, we use the transformation
    matrix to transfer MR label to CT-label
    
    fixed_img: CT Image
    moving_img: MR Image
    moving_label: MR label
    metric_method:
    ["extremity MRI"]: intra-subject; affine transformation, mutual information metric
    ["CTMR-based"]:intra-subject; multi-resolution (4)-
    rigid + B-spline transformation, Mutual Information metric (Mattes) with Adaptive Stochastic Gradient Descent optimizer
    
    We tranfer MR image/ MR label into CT space by initial transform and do the co-registration,
    the MR image and MR label spacing/origin will change and same with CT image
    """
    # Generate a centering transform based on the images
    IMAGE_DIMENSION = 3
    fixed_img = sitk.Cast(fixed_img, sitk.sitkFloat32)
    #fixed_img = adjust_array(fixed_img.GetDirection(),fixed_img)
    print(fixed_img.GetSize())
    fixed_flip = fixed_img

    moving_img = sitk.Cast(moving_img, sitk.sitkFloat32)
    #moving_img = adjust_array(moving_img.GetDirection(),moving_img)
    print(moving_img.GetSize())
    moving_flip = moving_img


    label_img = sitk.Cast(moving_label, sitk.sitkFloat32)
    #label_img = adjust_array(label_img.GetDirection(),label_img)
    print(label_img.GetSize())
    label_flip = label_img

    initial_transform = sitk.CenteredTransformInitializer(
        fixed_flip, # Note these are purposefuly reversed!
        moving_flip,# Note these are purposefuly reversed!
        sitk.Euler3DTransform(),
        sitk.CenteredTransformInitializerFilter.GEOMETRY)
    
    moving_resampled = sitk.Resample(
        moving_flip,       # Note these are purposefuly reversed!
        fixed_flip,      # Note these are purposefuly reversed!
        initial_transform,
        sitk.sitkNearestNeighbor,# TODO: use different interpolator?
        0.0,              # Note(Jacob): default value
        moving_flip.GetPixelID(),
    )
    label_resampled = sitk.Resample(
        label_flip,       # Note these are purposefuly reversed!
        fixed_flip,      # Note these are purposefuly reversed!
        initial_transform,
        sitk.sitkNearestNeighbor,# TODO: use different interpolator?
        0.0,              # Note(Jacob): default value
        label_flip.GetPixelID(),
    )
    print('Fixed size ', fixed_flip.GetSize())
    print('Moving size ', moving_flip.GetSize())
    print('Moving Resampled size', moving_resampled.GetSize())
    print('Label Resampled size', label_resampled.GetSize())

    print('Fixed space ', fixed_flip.GetSpacing())
    print('Moving space ', moving_flip.GetSpacing())
    print('Resampled space', moving_resampled.GetSpacing())
    print('Label Resampled space', label_resampled.GetSpacing())
    
    print('Fixed origin ', fixed_flip.GetOrigin())
    print('Moving origin ', moving_flip.GetOrigin())
    print('Resampled origin ', moving_resampled.GetOrigin())
    print('Label Resampled origin', label_resampled.GetOrigin())

    itk_fix = sitk2itk(fixed_flip,3)
    itk_mov = sitk2itk(moving_resampled,3)
    itk_label = sitk2itk(label_resampled,3)
    parameter_object = itk.ParameterObject.New()
    
#     if metric_method == 'extremity MRI':
#         parameter_object1.AddParameterFile(r'S:\StrokeAI\Try\co-regis\ElastixSources\Par0040affine.txt')
#     if metric_method == 'CTMR-based':
#         parameter_object1.AddParameterFile(r'S:\StrokeAI\Try\co-regis\ElastixSources\Par0043rigid.txt')
#     if metric_method == 'MRT1':
#         parameter_object1.AddParameterFile(r'S:\StrokeAI\Try\co-regis\ElastixSources\Par0002.fs.MI.rigid.RandomCoordinate.txt')
#     if metric_method == 'MRHASTE':
#         parameter_object1.AddParameterFile(r'S:\StrokeAI\Try\co-regis\ElastixSources\Par0010affine.txt')
#         parameter_object1.AddParameterFile(r'S:\StrokeAI\Try\co-regis\ElastixSources\Par0010bspline.txt')
#     parameter_object1.AddParameterFile(r'S:\StrokeAI\Try\co-regis\ElastixSources\Parameters_BSpline.txt')
    parameter_map_rigid1 = parameter_object.GetDefaultParameterMap('rigid')
    parameter_map_rigid1['FinalBSplineInterpolationOrder'] = ['0']
    parameter_object.AddParameterMap(parameter_map_rigid1)
#     parameter_map_affine1 = parameter_object.GetDefaultParameterMap('affine')
#     parameter_map_affine1['FinalBSplineInterpolationOrder'] = ['0']
#     parameter_object.AddParameterMap(parameter_map_affine1)
   # parameter_map_rigid2 = parameter_object1.GetDefaultParameterMap('rigid')
    #parameter_map_rigid2['FinalBSplineInterpolationOrder'] = ['0']
    #parameter_object1.AddParameterMap(parameter_map_rigid2)
    #parameter_map_affine2 = parameter_object1.GetDefaultParameterMap('affine')
   # parameter_map_affine2['FinalBSplineInterpolationOrder'] = ['0']
   # parameter_object1.AddParameterMap(parameter_map_affine2)
    #parameter_map_bspline1 = parameter_object1.GetDefaultParameterMap('bspline',4 )
    #parameter_map_bspline1['FinalBSplineInterpolationOrder'] = ['0']
    #parameter_object1.AddParameterMap(parameter_map_bspline1)
    #groupwise_parameter_map = parameter_object1.GetDefaultParameterMap('groupwise',4)
   # groupwise_parameter_map['FinalBSplineInterpolationOrder'] = ['0']
    #groupwise_parameter_map['Transform'] = ['EulerStackTransform']
  #  groupwise_parameter_map['AutomaticScalesEstimation'] = ['true']
  #  groupwise_parameter_map['AutomaticScalesEstimationStackTransform'] = ['true']
  #  parameter_object1.AddParameterMap(groupwise_parameter_map)
    #parameter_object2 = itk.ParameterObject.New()
    #parameter_map_bspline2 = parameter_object2.GetDefaultParameterMap('bspline')
    #parameter_object2.AddParameterMap(parameter_map_bspline2)
    #parameter_map_affine22 = parameter_object2.GetDefaultParameterMap('affine',2)
    #parameter_map_bspline2['FinalBSplineInterpolationOrder'] = ['0']
    #parameter_object2.AddParameterMap(parameter_map_bspline2)

#     parameter_map_bspline = parameter_object.GetDefaultParameterMap('bspline')
#     parameter_object.AddParameterMap(parameter_map_bspline)
    result_img, result_transform_parameters = itk.elastix_registration_method(itk_fix,itk_mov, parameter_object = parameter_object)
    print('itk_fix origin ',itk_fix.GetOrigin())
    print('itk_fix spacing ',itk_fix.GetSpacing())
    print('itk_mov origin ',itk_mov.GetOrigin())
    print('itk_mov spacing ',itk_mov.GetSpacing())
    print('result_img origin ',result_img.GetOrigin())
    print('result_img spacing ',result_img.GetSpacing())
    
    result_img = itk2sitk(result_img)
    print('result_img_transfer origin ',result_img.GetOrigin())
    print('result_img_transfer spacing ',result_img.GetSpacing())
    
    itk_label = sitk2itk(label_resampled,3)
    print('Label_origin origin ', itk_label.GetOrigin())
    print('Label_origin space ', itk_label.GetSpacing())
    
    transform_label_img1 =itk.transformix_filter(itk_label, result_transform_parameters)
    
    #result_img2, result_transform_parameters2 = itk.elastix_registration_method(itk_fix,result_img1, parameter_object = parameter_object2)
    #transform_label_img2 =itk.transformix_filter(transform_label_img1,result_transform_parameters2)

    label_result =itk2sitk(transform_label_img1)
    #result_img2 = itk2sitk(result_img2)

    print('Label_changed origin ', label_result.GetOrigin())
    print('Label_changed space ', label_result.GetSpacing())
    print('Label_changed size ', label_result.GetSize())
#     print('Label_changed size ', itk_label.GetSize())
    return label_result , result_img,label_resampled ,moving_resampled, result_transform_parameters, moving_flip, label_flip

## Test Case

In [ ]:
path = r"S:\StrokeAI\CT_Defaced_dataset\2244_1850602403\2244_1850602403_output.nii"
input_img = sitk.ReadImage(r"S:\StrokeAI\CT_Defaced_dataset\2236_3156947747\2249_1452006018_output.nii")
img = sitk.GetArrayFromImage(input_img)
img2 = CT_preprocess_final(path, [1.0, 1.0, 1.0] ,40, 80, (0.0,1.0),3,(0,0,0))
img_CT  = sitk.GetArrayFromImage(img2)

In [ ]:
os.chdir(r'S:\StrokeAI\CT_Defaced_dataset\2244_1850602403')
# fixed_resampled = sitk.GetImageFromArray(fixed_resampled)
sitk.WriteImage(img2,'2244_1850602403_test_preprocessed.nii')

In [29]:
#target_img = sitk.ReadImage("S:\StrokeAI\Try\co-regis\mri_test.nii.gz")
target_img = sitk.ReadImage(r"S:\StrokeAI\Stroke_Dataset\New_DataSet_MRI\2224_2564111995\89baf219_20171117_b0_MNI.nii.gz")
input_img = sitk.ReadImage(r"S:\StrokeAI\CT_Defaced_dataset\2224_2564111995\2224_2564111995_skullstrip.nii")
target_label =sitk.ReadImage(r'S:\StrokeAI\Stroke_Dataset\New_DataSet_MRI\2224_2564111995\89baf219_20171117_DAGMNet_CH3_Lesion_Predict_MNI.nii.gz')
#target_label =sitk.ReadImage(r'S:\StrokeAI\Try\co-regis\label_test.nii.gz')

In [47]:
label_result , result_img, result_transform_parameters,ct_img= coRegis_fix(input_img, target_img, target_label)


input_origin:  (98.0, 134.0, -72.0)
re_img size:  (197, 233, 189)
re_img spacing:  (1.0, 1.0, 1.0)
(98.0, 134.0, -72.0)
(-1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0)
(-1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0)
(197, 233, 189)
(197, 233, 189)
fixed_ori_direction: (-1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0)
moving_ori_direction:  (-1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0)
Fixed size  (197, 233, 189)
Moving size  (181, 217, 181)
Moving Resampled size (197, 233, 189)
Label Resampled size (197, 233, 189)
Fixed space  (1.0, 1.0, 1.0)
Moving space  (1.0, 1.0, 1.0)
Resampled space (1.0, 1.0, 1.0)
Label Resampled space (1.0, 1.0, 1.0)
Fixed origin  (98.0, 134.0, -72.0)
Moving origin  (89.0, 109.0, -91.0)
Moving Resampled origin  (98.0, 134.0, -72.0)
Label Resampled origin (98.0, 134.0, -72.0)
itk_fix origin  itkPointD3 ([98, 134, -72])
itk_fix_direction  itkMatrixD33 ([[-1.0, 0.0, 0.0], [0.0, -1.0, 0.0], [0.0, 0.0, 1.0]])
itk_fix spacing  itkVectorD3 ([1, 1, 1])
itk_mov origin  i

In [48]:
os.chdir('S:\StrokeAI\Try\co-regis')
   
#sitk.WriteImage(label_reampled,f'2224_2564111995_label_resampled.nii.gz')
#sitk.WriteImage(moving_resampled,f'2224_2564111995_mri_resampled.nii.gz')
sitk.WriteImage(label_result,f'2224_2564111995_label.nii.gz')
sitk.WriteImage(result_img,f'2224_2564111995_mri.nii.gz')
#sitk.WriteImage(moving_flip,f'2224_2564111995_moving_flip.nii.gz')
#sitk.WriteImage(label_flip,f'2224_2564111995_label_flip.nii.gz')
# os.makedirs(os.path.join(ct_dir,name),exist_ok = True)
# os.chdir(os.path.join(ct_dir,name))
sitk.WriteImage(ct_img,f'2224_2564111995_skullstrip.nii.gz')

In [ ]:
os.chdir('S:\StrokeAI\Try\co-regis')
# fixed_resampled = sitk.GetImageFromArray(fixed_resampled)
sitk.WriteImage(processed_ct,'0bb40770_20181012-test_processed_ct_skulloff.nii')

In [ ]:
origin_mri = sitk.GetArrayFromImage(target_img)
origin_label = sitk.GetArrayFromImage(target_label)
transfered_mri = sitk.GetArrayFromImage(sitk.ReadImage(r'S:\StrokeAI\Try\co-regis\test_resultimg_skulloff.nii'))
transfered_label = sitk.GetArrayFromImage(sitk.ReadImage(r'S:\StrokeAI\Try\co-regis\test_labelimg_skulloff.nii'))                                                   

In [ ]:
plt.subplot(1,2,1)
plt.imshow(origin_mri[75,:,:], cmap = "gray")
plt.imshow(origin_label[75,:,:],cmap = "Blues",alpha =0.3)
plt.show
plt.subplot(1,2,2)
plt.imshow(transfered_mri[70,:,:], cmap = "gray")
plt.imshow(transfered_label[70,:,:],cmap = "Blues",alpha =0.3)
plt.show

In [ ]:
origin = sitk.GetArrayFromImage(moving_resampled)

show_img(moving_resampled,0)

In [49]:
import pandas as pd
df=pd.read_csv('S:\StrokeAI\ctwmri_dataset.csv')
print(df.head())
ct_name = df['ct_dir_name']
mri_name = df['XNATSessionID']
id_dict = dict(zip(ct_name,mri_name))
for count, name in enumerate(ct_name):
    print(f'======{count+1}=======')
    print('ct_name: ',name)
    print('mri_name: ',mri_name[count])

    print('======================')

   Unnamed: 0                          mrn_hash  dos_rank      ct_dir_name  \
0           0  0dd4992d36fb4ba4f2891c82579b0560       1.0  2586_1651555515   
1           1  c0216f5bfbbe44d7eb1797f7255a591d       1.0  2352_2745484568   
2           2  53fd8b027a69d068dd79471daf32ee0b       1.0  2342_1809174254   
3           3  503eef98a82fb64894b4eace74c47379       1.0  2941_4521987789   
4           4  bf66c26ff73abd0628fdec0b53b82857       1.0  3017_8166192016   

       XNATSessionID                 ct_accession_hash  \
0  3e35b285_20200910  aefc0547480dac5c04fde20586823c81   
1  6e620108_20171223  c91f2e17b33d1eb3ff7d524f46c3ed6c   
2  46361d46_20200308  8e3d242990f79f41cdff4a7984a761ce   
3  d3c9372b_20181214  0bf1368a8b53999fee0a806a738557dd   
4  3c74d02d_20180129  d4e1b9dff92d6216dd5ed6694e119ae4   

                  mr_accession_hash  ct_mr_delta_hours  
0  f02e6a6a3e6569071538bd2b0454717a               38.0  
1  9ca20b6c494bcc30442ea0c5a45854da               13.0  
2  eed7957d

In [50]:
sorted(id_dict.keys())

['2224_2564111995',
 '2225_2564111995',
 '2226_2061339079',
 '2227_2339850822',
 '2228_1836983617',
 '2230_7063720593',
 '2231_3082828075',
 '2234_1406879670',
 '2235_8257173982',
 '2236_3156947747',
 '2237_4562088604',
 '2240_1854400938',
 '2241_1117341842',
 '2243_7049095170',
 '2244_1850602403',
 '2247_1399604056',
 '2249_1452006018',
 '2250_3398157536',
 '2251_1831897518',
 '2254_8113447932',
 '2255_4875927474',
 '2258_3145035410',
 '2261_2510573068',
 '2262_3233827331',
 '2267_2599225457',
 '2268_1728202471',
 '2270_3425522123',
 '2271_3559322004',
 '2273_3758915255',
 '2275_1094006870',
 '2276_2782679189',
 '2278_6799124093',
 '2280_2532845350',
 '2281_2115049809',
 '2282_6296583556',
 '2284_9701091364',
 '2286_3004295140',
 '2287_6198187275',
 '2288_3361948660',
 '2289_1546144185',
 '2290_3173569486',
 '2293_2958402408',
 '2295_3569527892',
 '2297_5482867529',
 '2298_9710048992',
 '2299_2155688317',
 '2300_1109689758',
 '2301_1090672398',
 '2302_5202280354',
 '2304_5260553616',


In [51]:
ct_name = sorted(df['ct_dir_name'])
len(ct_name)

603

In [59]:
%%time
df=pd.read_csv('S:\StrokeAI\ctwmri_dataset.csv')
ct_name = df['ct_dir_name']
ct_name = sorted(ct_name)
sor_dir = r'S:\StrokeAI\Stroke_Dataset'
origin_ctdir = r'S:\StrokeAI\CT_Defaced_dataset'
origin_mridir = r'S:\StrokeAI\Stroke_Dataset\New_DataSet_MRI'
ct_dir = r'S:\StrokeAI\Stroke_Dataset\New_CT'
mri_dir = r'S:\StrokeAI\Stroke_Dataset\New_MRI'
incomplete = []
for count, name in enumerate(ct_name):

    ori_ctpatdir = os.path.join(origin_ctdir,name)
    ori_mripatdir = os.path.join(origin_mridir,name)
    print(f'============{count+1}=============')
    try:
        #copy ct finalpreprocessed file into StrokeAI_Dataset/CT/name
        #ctfile = os.path.join(ori_ctpatdir,f'{name}_finalpreprocessed.nii')
        ctskulloff = os.path.join(ori_ctpatdir,f'{name}_skullstrip.nii')
        target_fixed = sitk.ReadImage(ctskulloff)
        print('ct_img: ',ctskulloff)
        mri_img = os.path.join(ori_mripatdir,f'{id_dict[f"{name}"]}_b0_MNI.nii.gz')
        target_moving = sitk.ReadImage(mri_img)
        label_img = os.path.join(ori_mripatdir,f'{id_dict[f"{name}"]}_DAGMNet_CH3_Lesion_Predict_MNI.nii.gz')
        target_label = sitk.ReadImage(label_img)
        label_result , result_img, result_transform_parameters,ct_img= coRegis_fix(target_fixed, target_moving, target_label)

        
        #CT skulltrip file into StrokeAI_Dataset/New_CT/name
        os.makedirs(os.path.join(ct_dir,name),exist_ok = True)
        os.chdir(os.path.join(ct_dir,name))
        save_path = os.path.join(ct_dir,name)
        sitk.WriteImage(ct_img,f'{name}_ct.nii.gz')
        print('CT_file: ',os.listdir(save_path))
        
        #MRI file into StrokeAI_Dataset/New_MRI/name
        os.makedirs(os.path.join(mri_dir,name),exist_ok = True)
        os.chdir(os.path.join(mri_dir,name))
        save_path = os.path.join(mri_dir,name)
        sitk.WriteImage(result_img,f'{name}_b0_MNI_coRegist.nii.gz')
        sitk.WriteImage(label_result,f'{name}_label_MNI_coRegist.nii.gz')
        print('MRI_file: ',os.listdir(save_path))
        
        #target_img = LPI2RAS(os.path.join(ori_mripatdir,f'{id_dict[f"{name}"]}_b0_MNI.nii.gz'),os.path.join(save_paht,f'{id_dict[f"{name}"]}_b0_MNI.nii.gz'))
        #target_label = LPI2RAS(os.path.join(ori_mripatdir,f'{id_dict[f"{name}"]}_DAGMNet_CH3_Lesion_Predict_MNI.nii.gz'),os.path.join(save_paht,f'{id_dict[f"{name}"]}_DAGMNet_CH3_Lesion_Predict_MNI.nii.gz'))

         #target_img = sitk.ReadImage(os.path.join(ori_mripatdir,f'{id_dict[f"{name}"]}_b0_MNI.nii.gz'))
        #print('mir_img: ',os.path.join(ori_mripatdir,f'{id_dict[f"{name}"]}_b0_MNI.nii.gz'))
        #print('ct_img_skulloff: ',ctskulloff)
        #input_img = sitk.ReadImage(ctskulloff)
        #target_label = sitk.ReadImage(os.path.join(ori_mripatdir,f'{id_dict[f"{name}"]}_DAGMNet_CH3_Lesion_Predict_MNI.nii.gz'))
        #print('mir_label: ',os.path.join(ori_mripatdir,f'{id_dict[f"{name}"]}_DAGMNet_CH3_Lesion_Predict_MNI.nii.gz'))
        #label_result, result_img,label_reampled,moving_resampled,tf, moving_flip, label_flip,processed_ct= coRegis_final(input_img, target_img, target_label,"None")
#         label_result = sitk.GetArrayFromImage(label_result)
# #         label_result[label_result<1e-2] = 0.0
# #         label_result[label_result>0.0] = 1.0
#         label_result = sitk.GetImageFromArray(label_result)

#         result_img = sitk.GetArrayFromImage(result_img)
#         result_img[result_img<0] = 0.0
#         result_img = sitk.GetImageFromArray(result_img)
        #sitk.WriteImage(label_reampled,f'{name}_label_resampled.nii.gz')
        #sitk.WriteImage(moving_resampled,f'{name}_mri_resampled.nii.gz')
        #sitk.WriteImage(label_result,f'{name}_label.nii.gz')
        #sitk.WriteImage(result_img,f'{name}_mri.nii.gz')
        #sitk.WriteImage(moving_flip,f'{name}_moving_flip.nii.gz')
        #sitk.WriteImage(label_flip,f'{name}_label_flip.nii.gz')
        #os.makedirs(os.path.join(ct_dir,name),exist_ok = True)
        #os.chdir(os.path.join(ct_dir,name))
        #sitk.WriteImage(processed_ct,f'{name}_skullstrip.nii.gz')

    except :
        incomplete.append(name)
    
    print("=================================")

print('========Dataset-Generated=======')

============1=============
ct_img:  S:\StrokeAI\CT_Defaced_dataset\2224_2564111995\2224_2564111995_skullstrip.nii
input_origin:  (98.0, 134.0, -72.0)
re_img size:  (197, 233, 189)
re_img spacing:  (1.0, 1.0, 1.0)
(98.0, 134.0, -72.0)
(-1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0)
(-1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0)
(197, 233, 189)
(197, 233, 189)
fixed_ori_direction: (-1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0)
moving_ori_direction:  (-1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0)
Fixed size  (197, 233, 189)
Moving size  (181, 217, 181)
Moving Resampled size (197, 233, 189)
Label Resampled size (197, 233, 189)
Fixed space  (1.0, 1.0, 1.0)
Moving space  (1.0, 1.0, 1.0)
Resampled space (1.0, 1.0, 1.0)
Label Resampled space (1.0, 1.0, 1.0)
Fixed origin  (98.0, 134.0, -72.0)
Moving origin  (89.0, 109.0, -91.0)
Moving Resampled origin  (98.0, 134.0, -72.0)
Label Resampled origin (98.0, 134.0, -72.0)
itk_fix origin  itkPointD3 ([98, 134, -72])
itk_fix_direction  itkMatrixD33 

ct_img:  S:\StrokeAI\CT_Defaced_dataset\2228_1836983617\2228_1836983617_skullstrip.nii
input_origin:  (98.0, 134.0, -72.0)
re_img size:  (197, 233, 189)
re_img spacing:  (1.0, 1.0, 1.0)
(98.0, 134.0, -72.0)
(-1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0)
(-1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0)
(197, 233, 189)
(197, 233, 189)
fixed_ori_direction: (-1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0)
moving_ori_direction:  (-1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0)
Fixed size  (197, 233, 189)
Moving size  (181, 217, 181)
Moving Resampled size (197, 233, 189)
Label Resampled size (197, 233, 189)
Fixed space  (1.0, 1.0, 1.0)
Moving space  (1.0, 1.0, 1.0)
Resampled space (1.0, 1.0, 1.0)
Label Resampled space (1.0, 1.0, 1.0)
Fixed origin  (98.0, 134.0, -72.0)
Moving origin  (89.0, 109.0, -91.0)
Moving Resampled origin  (98.0, 134.0, -72.0)
Label Resampled origin (98.0, 134.0, -72.0)
itk_fix origin  itkPointD3 ([98, 134, -72])
itk_fix_direction  itkMatrixD33 ([[-1.0, 0.0, 0.0], [0.0, -

ct_img:  S:\StrokeAI\CT_Defaced_dataset\2235_8257173982\2235_8257173982_skullstrip.nii
input_origin:  (98.0, 134.0, -72.0)
re_img size:  (197, 233, 189)
re_img spacing:  (1.0, 1.0, 1.0)
(98.0, 134.0, -72.0)
(-1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0)
(-1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0)
(197, 233, 189)
(197, 233, 189)
fixed_ori_direction: (-1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0)
moving_ori_direction:  (-1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0)
Fixed size  (197, 233, 189)
Moving size  (181, 217, 181)
Moving Resampled size (197, 233, 189)
Label Resampled size (197, 233, 189)
Fixed space  (1.0, 1.0, 1.0)
Moving space  (1.0, 1.0, 1.0)
Resampled space (1.0, 1.0, 1.0)
Label Resampled space (1.0, 1.0, 1.0)
Fixed origin  (98.0, 134.0, -72.0)
Moving origin  (89.0, 109.0, -91.0)
Moving Resampled origin  (98.0, 134.0, -72.0)
Label Resampled origin (98.0, 134.0, -72.0)
itk_fix origin  itkPointD3 ([98, 134, -72])
itk_fix_direction  itkMatrixD33 ([[-1.0, 0.0, 0.0], [0.0, -

ct_img:  S:\StrokeAI\CT_Defaced_dataset\2241_1117341842\2241_1117341842_skullstrip.nii
input_origin:  (98.0, 134.0, -72.0)
re_img size:  (197, 233, 189)
re_img spacing:  (1.0, 1.0, 1.0)
(98.0, 134.0, -72.0)
(-1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0)
(-1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0)
(197, 233, 189)
(197, 233, 189)
fixed_ori_direction: (-1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0)
moving_ori_direction:  (-1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0)
Fixed size  (197, 233, 189)
Moving size  (181, 217, 181)
Moving Resampled size (197, 233, 189)
Label Resampled size (197, 233, 189)
Fixed space  (1.0, 1.0, 1.0)
Moving space  (1.0, 1.0, 1.0)
Resampled space (1.0, 1.0, 1.0)
Label Resampled space (1.0, 1.0, 1.0)
Fixed origin  (98.0, 134.0, -72.0)
Moving origin  (89.0, 109.0, -91.0)
Moving Resampled origin  (98.0, 134.0, -72.0)
Label Resampled origin (98.0, 134.0, -72.0)
itk_fix origin  itkPointD3 ([98, 134, -72])
itk_fix_direction  itkMatrixD33 ([[-1.0, 0.0, 0.0], [0.0, -

ct_img:  S:\StrokeAI\CT_Defaced_dataset\2249_1452006018\2249_1452006018_skullstrip.nii
input_origin:  (98.0, 134.0, -72.0)
re_img size:  (197, 233, 189)
re_img spacing:  (1.0, 1.0, 1.0)
(98.0, 134.0, -72.0)
(-1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0)
(-1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0)
(197, 233, 189)
(197, 233, 189)
fixed_ori_direction: (-1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0)
moving_ori_direction:  (-1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0)
Fixed size  (197, 233, 189)
Moving size  (181, 217, 181)
Moving Resampled size (197, 233, 189)
Label Resampled size (197, 233, 189)
Fixed space  (1.0, 1.0, 1.0)
Moving space  (1.0, 1.0, 1.0)
Resampled space (1.0, 1.0, 1.0)
Label Resampled space (1.0, 1.0, 1.0)
Fixed origin  (98.0, 134.0, -72.0)
Moving origin  (89.0, 109.0, -91.0)
Moving Resampled origin  (98.0, 134.0, -72.0)
Label Resampled origin (98.0, 134.0, -72.0)
itk_fix origin  itkPointD3 ([98, 134, -72])
itk_fix_direction  itkMatrixD33 ([[-1.0, 0.0, 0.0], [0.0, -

...

ct_img:  S:\StrokeAI\CT_Defaced_dataset\3221_7061316737\3221_7061316737_skullstrip.nii
input_origin:  (98.0, 134.0, -72.0)
re_img size:  (197, 233, 189)
re_img spacing:  (1.0, 1.0, 1.0)
(98.0, 134.0, -72.0)
(-1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0)
(-1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0)
(197, 233, 189)
(197, 233, 189)
fixed_ori_direction: (-1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0)
moving_ori_direction:  (-1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0)
Fixed size  (197, 233, 189)
Moving size  (181, 217, 181)
Moving Resampled size (197, 233, 189)
Label Resampled size (197, 233, 189)
Fixed space  (1.0, 1.0, 1.0)
Moving space  (1.0, 1.0, 1.0)
Resampled space (1.0, 1.0, 1.0)
Label Resampled space (1.0, 1.0, 1.0)
Fixed origin  (98.0, 134.0, -72.0)
Moving origin  (89.0, 109.0, -91.0)
Moving Resampled origin  (98.0, 134.0, -72.0)
Label Resampled origin (98.0, 134.0, -72.0)
itk_fix origin  itkPointD3 ([98, 134, -72])
itk_fix_direction  itkMatrixD33 ([[-1.0, 0.0, 0.0], [0.0, -

ct_img:  S:\StrokeAI\CT_Defaced_dataset\3230_1880515665\3230_1880515665_skullstrip.nii
input_origin:  (98.0, 134.0, -72.0)
re_img size:  (197, 233, 189)
re_img spacing:  (1.0, 1.0, 1.0)
(98.0, 134.0, -72.0)
(-1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0)
(-1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0)
(197, 233, 189)
(197, 233, 189)
fixed_ori_direction: (-1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0)
moving_ori_direction:  (-1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0)
Fixed size  (197, 233, 189)
Moving size  (181, 217, 181)
Moving Resampled size (197, 233, 189)
Label Resampled size (197, 233, 189)
Fixed space  (1.0, 1.0, 1.0)
Moving space  (1.0, 1.0, 1.0)
Resampled space (1.0, 1.0, 1.0)
Label Resampled space (1.0, 1.0, 1.0)
Fixed origin  (98.0, 134.0, -72.0)
Moving origin  (89.0, 109.0, -91.0)
Moving Resampled origin  (98.0, 134.0, -72.0)
Label Resampled origin (98.0, 134.0, -72.0)
itk_fix origin  itkPointD3 ([98, 134, -72])
itk_fix_direction  itkMatrixD33 ([[-1.0, 0.0, 0.0], [0.0, -

ct_img:  S:\StrokeAI\CT_Defaced_dataset\3245_3077873151\3245_3077873151_skullstrip.nii
input_origin:  (98.0, 134.0, -72.0)
re_img size:  (197, 233, 189)
re_img spacing:  (1.0, 1.0, 1.0)
(98.0, 134.0, -72.0)
(-1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0)
(-1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0)
(197, 233, 189)
(197, 233, 189)
fixed_ori_direction: (-1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0)
moving_ori_direction:  (-1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0)
Fixed size  (197, 233, 189)
Moving size  (181, 217, 181)
Moving Resampled size (197, 233, 189)
Label Resampled size (197, 233, 189)
Fixed space  (1.0, 1.0, 1.0)
Moving space  (1.0, 1.0, 1.0)
Resampled space (1.0, 1.0, 1.0)
Label Resampled space (1.0, 1.0, 1.0)
Fixed origin  (98.0, 134.0, -72.0)
Moving origin  (89.0, 109.0, -91.0)
Moving Resampled origin  (98.0, 134.0, -72.0)
Label Resampled origin (98.0, 134.0, -72.0)
itk_fix origin  itkPointD3 ([98, 134, -72])
itk_fix_direction  itkMatrixD33 ([[-1.0, 0.0, 0.0], [0.0, -

ct_img:  S:\StrokeAI\CT_Defaced_dataset\3252_1370318460\3252_1370318460_skullstrip.nii
input_origin:  (98.0, 134.0, -72.0)
re_img size:  (197, 233, 189)
re_img spacing:  (1.0, 1.0, 1.0)
(98.0, 134.0, -72.0)
(-1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0)
(-1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0)
(197, 233, 189)
(197, 233, 189)
fixed_ori_direction: (-1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0)
moving_ori_direction:  (-1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0)
Fixed size  (197, 233, 189)
Moving size  (181, 217, 181)
Moving Resampled size (197, 233, 189)
Label Resampled size (197, 233, 189)
Fixed space  (1.0, 1.0, 1.0)
Moving space  (1.0, 1.0, 1.0)
Resampled space (1.0, 1.0, 1.0)
Label Resampled space (1.0, 1.0, 1.0)
Fixed origin  (98.0, 134.0, -72.0)
Moving origin  (89.0, 109.0, -91.0)
Moving Resampled origin  (98.0, 134.0, -72.0)
Label Resampled origin (98.0, 134.0, -72.0)
itk_fix origin  itkPointD3 ([98, 134, -72])
itk_fix_direction  itkMatrixD33 ([[-1.0, 0.0, 0.0], [0.0, -

ct_img:  S:\StrokeAI\CT_Defaced_dataset\3258_8168310579\3258_8168310579_skullstrip.nii
input_origin:  (98.0, 134.0, -72.0)
re_img size:  (197, 233, 189)
re_img spacing:  (1.0, 1.0, 1.0)
(98.0, 134.0, -72.0)
(-1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0)
(-1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0)
(197, 233, 189)
(197, 233, 189)
fixed_ori_direction: (-1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0)
moving_ori_direction:  (-1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0)
Fixed size  (197, 233, 189)
Moving size  (181, 217, 181)
Moving Resampled size (197, 233, 189)
Label Resampled size (197, 233, 189)
Fixed space  (1.0, 1.0, 1.0)
Moving space  (1.0, 1.0, 1.0)
Resampled space (1.0, 1.0, 1.0)
Label Resampled space (1.0, 1.0, 1.0)
Fixed origin  (98.0, 134.0, -72.0)
Moving origin  (89.0, 109.0, -91.0)
Moving Resampled origin  (98.0, 134.0, -72.0)
Label Resampled origin (98.0, 134.0, -72.0)
itk_fix origin  itkPointD3 ([98, 134, -72])
itk_fix_direction  itkMatrixD33 ([[-1.0, 0.0, 0.0], [0.0, -

In [57]:
CT_MR_Based = incomplete

In [58]:
print(CT_MR_Based)

[]


In [ ]:
len(ct_name)

In [ ]:
incomplete2 = incomplete
ExtremMRI = incomplete2

In [ ]:
need_copy = sorted(list(set(ct_name) - set(CT_MR_Based)))

In [ ]:
incomplete3 = incomplete
MRHASTE = incomplete3

In [ ]:
need_copy = """
'2224_2564111995',
 '2225_2564111995',
 '2226_2061339079',
 '2227_2339850822',
 '2228_1836983617',
 '2230_7063720593',
 '2231_3082828075',
 '2234_1406879670',
 '2235_8257173982',
 '2236_3156947747',
 '2237_4562088604',
 '2240_1854400938',
 '2241_1117341842',
 '2243_7049095170',
 '2244_1850602403',
 '2247_1399604056',
 '2249_1452006018',
 '2250_3398157536',
 '2251_1831897518',
 '2254_8113447932',
 '2255_4875927474',
 '2258_3145035410',
 '2261_2510573068',
 '2262_3233827331',
 '2267_2599225457',
 '2268_1728202471',
 '2270_3425522123',
 '2271_3559322004',
 '2273_3758915255',
 '2275_1094006870',
 '2276_2782679189',
 '2278_6799124093',
 '2280_2532845350',
 '2281_2115049809',
 '2282_6296583556',
 '2284_9701091364',
 '2286_3004295140',
 '2287_6198187275',
 '2288_3361948660',
 '2289_1546144185',
 '2290_3173569486',
 '2293_2958402408',
 '2295_3569527892',
 '2297_5482867529',
 '2298_9710048992',
 '2299_2155688317',
 '2300_1109689758',
 '2301_1090672398',
 '2304_5260553616',
 '2308_3251134270',
 '2309_8221611558',
 '2310_2030497762',
 '2314_1849362056',
 '2315_8299985582',
 '2316_1053866790',
 '2317_1253570929',
 '2319_2240420277',
 '2320_2531453430',
 '2321_1066986767',
 '2323_1558647193',
 '2324_1047084599',
 '2325_6360025339',
 '2326_1312316808',
 '2327_2754984454',
 '2329_1317555555',
 '2331_1523652985',
 '2332_1332810907',
 '2338_3309211667',
 '2339_3752174485',
 '2341_6506786851',
 '2342_1809174254',
 '2343_2898251269',
 '2344_1784693666',
 '2347_1406539650',
 '2348_1224379480',
 '2350_2505780016',
 '2351_1480275434',
 '2352_2745484568',
 '2354_2979567507',
 '2356_1816582544',
 '2357_2422928533',
 '2358_3179903506',
 '2359_1143641672',
 '2361_2930333975',
 '2362_2086466981',
 '2364_3003975353',
 '2365_5011748479',
 '2367_1070928669',
 '2369_2402309829',
 '2370_2617539245',
 '2371_4977420715',
 '2373_9773454345',
 '2375_2689052835',
 '2378_3876451394',
 '2383_3311163278',
 '2384_2432006578',
 '2385_2633461759',
 '2386_1583316811',
 '2387_2633461759',
 '2388_2633461759',
 '2389_2716809323',
 '2390_3257740539',
 '2392_3387008770',
 '2393_1579978364',
 '2395_2969430364',
 '2396_1611014548',
 '2398_9065013941',
 '2399_5459176951',
 '2402_1511097630',
 '2404_1540277513',
 '2407_3218921773',
 '2408_1130109742',
 '2410_2045793529',
 '2411_1126762255',
 '2415_1832380557',
 '2416_5966879525',
 '2417_2535096532',
 '2419_1718356596',
 '2423_1179374067',
 '2424_1915094046',
 '2426_2147877392',
 '2427_3251191874',
 '2428_3281428767',
 '2430_1149859662',
 '2431_1281287925',
 '2432_3338125403',
 '2435_2272368435',
 '2436_9697792254',
 '2437_6841614189',
 '2438_3089122303',
 '2439_1915212597',
 '2440_1915212597',
 '2447_2730690840',
 '2449_2310756073',
 '2451_1306393952',
 '2452_1469111226',
 '2453_2118179142',
 '2457_3360704329',
 '2458_9093467395',
 '2460_3329191953',
 '2461_2767934036',
 '2464_1134400670',
 '2465_3393111926',
 '2466_3294437236',
 '2467_5848318514',
 '2470_3385353546',
 '2472_1401034787',
 '2475_3222867941',
 '2476_4528627326',
 '2477_2365258310',
 '2478_7354279871',
 '2480_1623205057',
 '2481_1989125293',
 '2486_3030790028',
 '2488_2262220363',
 '2490_8361455010',
 '2491_2836852095',
 '2493_1737811408',
 '2494_1603583309',
 '2495_2808635939',
 '2496_3571065644',
 '2497_3013694280',
 '2498_1612606151',
 '2501_2489790710',
 '2507_2431828693',
 '2509_1127551938',
 '2510_2264922808',
 '2511_7748937238',
 '2512_9797248346',
 '2513_2987033001',
 '2514_3030010822',
 '2516_7691783281',
 '2517_1987048534',
 '2519_1328878379',
 '2521_2658473873',
 '2523_9119221861',
 '2526_2638809883',
 '2527_2554974099',
 '2528_2778652941',
 '2529_1385534111',
 '2531_1696667171',
 '2532_2731440938',
 '2533_1633204258',
 '2534_1889987918',
 '2535_9324685971',
 '2537_3059055122',
 '2538_3367098122',
 '2543_2894800801',
 '2544_3238972517',
 '2546_3105040837',
 '2547_1545690387',
 '2553_3325959404',
 '2554_2085286941',
 '2555_7398625095',
 '2557_3576695675',
 '2558_2600679528',
 '2560_1622558335',
 '2562_2231578854',
 '2565_1067898584',
 '2566_1329019832',
 '2567_9427278888',
 '2568_9034607140',
 '2569_6740540163',
 '2570_1382606866',
 '2571_1880061564',
 '2574_9429834720',
 '2575_1165750622',
 '2577_1692005265',
 '2579_6736445705',
 '2581_2033916899',
 '2584_1297814680',
 '2586_1651555515',
 '2587_2107318866',
 '2589_3389945183',
 '2590_1240376081',
 '2592_7804356343',
 '2594_1193594639',
 '2595_2315387681',
 '2596_3172035407',
 '2598_2314947813',
 '2599_8317581228',
 '2601_2983469537',
 '2603_3239772719',
 '2606_2999160022',
 '2608_1568780569',
 '2611_1305726392',
 '2612_2243480327',
 '2613_2531023981',
 '2615_1100995162',
 '2616_2507861988',
 '2618_1025905440',
 '2619_7747326685',
 '2620_3076326067',
 '2621_2598965783',
 '2622_2136932176',
 '2623_1350376159',
 '2627_1494856751',
 '2628_2504968139',
 '2629_1998180782',
 '2630_2601321159',
 '2634_1884535515',
 '2637_2086969724',
 '2638_1634566566',
 '2639_1492006128',
 '2643_1666973011',
 '2644_1444627662',
 '2646_1816150506',
 '2647_3383186867',
 '2649_2302906918',
 '2650_2209210968',
 '2651_7108378507',
 '2652_2901254859',
 '2653_9003070470',
 '2654_4804852375',
 '2657_2134840776',
 '2658_2423620063',
 '2660_2225913571',
 '2662_8797322952',
 '2663_3652734274',
 '2664_1901922283',
 '2665_1277653488',
 '2666_1952913604',
 '2667_1917712169',
 '2668_2112702020',
 '2669_1443109235',
 '2670_2316852982',
 '2671_1296617584',
 '2674_1863321166',
 '2675_1814232703',
 '2677_3021453942',
 '2679_2642514464',
 '2683_1003443553',
 '2685_8280245295',
 '2687_2084460717',
 '2688_2666937458',
 '2690_6556621310',
 '2691_3247450148',
 '2692_2918909279',
 '2696_2020792178',
 '2697_1845630889',
 '2698_1886845421',
 '2700_2514234346',
 '2701_2291886930',
 '2702_1697056466',
 '2704_2791531954',
 '2706_2776809278',
 '2708_1111260257',
 '2709_1925881014',
 '2710_3369612708',
 '2717_2974869217',
 '2718_4576579659',
 '2719_2828977330',
 '2720_1407821446',
 '2723_1516748921',
 '2724_1110778744',
 '2726_1529205650',
 '2727_1205525128',
 '2732_2037156306',
 '2759_3350193834',
 '2763_9716457986',
 '2764_1688103997',
 '2766_2088427482',
 '2768_1276237715',
 '2770_3321096925',
 '2771_7464971735',
 '2772_2432242505',
 '2773_3276062983',
 '2774_1683246059',
 '2777_6083514629',
 '2778_1827234964',
 '2780_3361285289',
 '2781_2811353044',
 '2784_1624366498',
 '2787_1459632872',
 '2788_1210405155',
 '2789_3220664538',
 '2791_1103562044',
 '2792_2137363722',
 '2793_2650477387',
 '2794_1825897299',
 '2795_2093756907',
 '2797_1931012584',
 '2799_1932550997',
 '2800_3250062171',
 '2801_1008909290',
 '2803_3187550483',
 '2804_1240212543',
 '2805_2677966376',
 '2806_1395751628',
 '2809_3045739029',
 '2810_2252879660',
 '2811_2252879660',
 '2812_2252879660',
 '2813_3017119396',
 '2816_2252879660',
 '2817_2106881455',
 '2819_5924348427',
 '2821_3349244093',
 '2822_6500498763',
 '2823_2386872545',
 '2827_1016053672',
 '2828_2966081470',
 '2829_7099488441',
 '2830_1016053672',
 '2831_9411296777',
 '2832_1244214139',
 '2834_1919549476',
 '2835_1758919143',
 '2837_1094836480',
 '2840_2982826561',
 '2842_1983941032',
 '2843_1238120029',
 '2845_1238120029',
 '2846_1880045085',
 '2848_3020767660',
 '2849_2454664036',
 '2851_1782660844',
 '2852_3335675312',
 '2853_2578641922',
 '2854_3247004977',
 '2855_8010018745',
 '2856_3730567245',
 '2857_8636282609',
 '2859_3119399027',
 '2861_1070961337',
 '2865_4101638014',
 '2867_2180416023',
 '2879_6740902178',
 '2880_1088865999',
 '2884_1862461178',
 '2885_5905128622',
 '2886_3228072470',
 '2887_3228072470',
 '2888_1406602716',
 '2889_1068317435',
 '2890_1625571752',
 '2891_1447696048',
 '2892_9749971548',
 '2893_9521938156',
 '2895_1065242900',
 '2897_2953316208',
 '2898_8821020467',
 '2899_2880246190',
 '2900_1319200512',
 '2901_2809501442',
 '2902_1176333340',
 '2904_4633915158',
 '2906_1800747315',
 '2907_1240407509',
 '2908_2614300284',
 '2909_2752980547',
 '2910_1081957885',
 '2911_3225802173',
 '2913_2149557883',
 '2914_2156556011',
 '2915_2156556011',
 '2916_2116050369',
 '2917_1409289234',
 '2920_1667288728',
 '2926_1169046567',
 '2927_3028969190',
 '2928_5247900315',
 '2929_2148502241',
 '2930_7573474978',
 '2933_7791595791',
 '2935_3013757537',
 '2937_1513549944',
 '2939_4568755518',
 '2940_7558031110',
 '2941_4521987789',
 '2942_3045044732',
 '2944_2777923638',
 '2945_2813328895',
 '2946_1625191686',
 '2947_1057783105',
 '2948_1619679856',
 '2955_2423763783',
 '2958_2240335119',
 '2959_1332390580',
 '2960_1560222156',
 '2961_2181144906',
 '2963_1546062524',
 '2967_1494694724',
 '2970_1721663744',
 '2972_1557477071',
 '2975_2074588655',
 '2976_2074588655',
 '2977_9329249264',
 '2978_1322904889',
 '2979_3302667984',
 '2983_2320390125',
 '2984_3224974737',
 '2985_1639249211',
 '2987_2776979947',
 '2989_3385334029',
 '2992_1457591804',
 '2996_3186208397',
 '2998_2324054264',
 '3000_2801409266',
 '3001_2080765608',
 '3003_2080765608',
 '3005_3246570339',
 '3009_2356332190',
 '3011_2400677505',
 '3012_6559622978',
 '3013_3023181789',
 '3015_6087632858',
 '3016_1185814198',
 '3017_8166192016',
 '3018_6087632858',
 '3019_2970797152',
 '3025_8176013810',
 '3026_5896469473',
 '3029_7434845353',
 '3032_9027744163',
 '3034_2442413736',
 '3039_2105901755',
 '3040_2928053167',
 '3044_1936552041',
 '3046_3379772483',
 '3047_3165566635',
 '3048_1638422859',
 '3049_2335347430',
 '3050_7931593944',
 '3051_2062011371',
 '3053_3473303861',
 '3054_8220488592',
 '3055_1063224789',
 '3056_2309409035',
 '3057_2385363386',
 '3058_1829908648',
 '3060_6179677284',
 '3065_1224593531',
 '3066_6933624153',
 '3069_1691602205',
 '3070_1254269019',
 '3071_1698956539',
 '3072_2082776788',
 '3073_1730775310',
 '3074_5729614042',
 '3075_2828460585',
 '3077_5874093320',
 '3078_3367251286',
 '3079_3128962761',
 '3081_2403137140',
 '3082_1084738807',
 '3083_1084738807',
 '3085_1162481095',
 '3086_5730839906',
 '3087_7102507607',
 '3089_1793592715',
 '3091_2651190085',
 '3092_4317610292',
 '3094_1643689422',
 '3095_6466223309',
 '3096_1902362598',
 '3097_3198183550',
 '3098_1668639882',
 '3099_1902362598',
 '3101_1894929448',
 '3103_2598392163',
 '3106_1796587439',
 '3107_1754370967',
 '3108_3163853045',
 '3110_1754370967',
 '3112_4872611097',
 '3113_3027387326',
 '3114_1246519828',
 '3115_9158661564',
 '3120_6678495563',
 '3121_4580392102',
 '3122_2826348398',
 '3124_2236831900',
 '3125_1408706537',
 '3126_9209508462',
 '3127_3100157562',
 '3128_1643331351',
 '3131_3025553059',
 '3132_1387419488',
 '3133_2446865357',
 '3135_2215979253',
 '3136_2215979253',
 '3138_2215979253',
 '3139_2848614875',
 '3140_8571747425',
 '3152_2735818692',
 '3157_1262145452',
 '3158_2675577253',
 '3160_9217038961',
 '3164_2438790103',
 '3167_5558641423',
 '3168_1170583330',
 '3170_1064421139',
 '3185_2467880414',
 '3188_3256079554',
 '3195_8851502224',
 '3196_8786756653',
 '3198_1460876043',
 '3200_1652774501',
 '3202_2347401147',
 '3203_2347401147',
 '3211_2092219175',
 '3212_1126826426',
 '3214_2939653037',
 '3216_2683576144',
 '3217_1313747721',
 '3218_9004580657',
 '3219_3218178668',
 '3220_2935115578',
 '3221_7061316737',
 '3224_2102276195',
 '3225_1323765026',
 '3226_3199198242',
 '3230_1880515665',
 '3233_3285280025',
 '3240_1733211757',
 '3244_2009155265',
 '3245_3077873151',
 '3246_2288089785',
 '3248_1731019636',
 '3249_5217994659',
 '3252_1370318460',
 '3255_8605684186',
 '3257_9018609989'
 """.split(',')

In [ ]:
len(need_copy)

In [ ]:
import glob
ct_transfer = []
mri_transfer = []
for ct in glob.glob(r'S:\StrokeAI\Stroke_Dataset\CT\*'):
    ct_transfer.append(os.path.split(ct)[-1])
for mri in glob.glob(r'S:\StrokeAI\Stroke_Dataset\MRI\*'):
    mri_transfer.append(os.path.split(mri)[-1])

In [ ]:
ct_transfer = sorted(ct_transfer)
mri_transfer = sorted(mri_transfer)
set(mri_transfer) - set(ct_transfer)

In [ ]:
import pandas as pd
df=pd.read_csv('S:\StrokeAI\ctwmri_dataset_1days.csv')
print(df.head())
ct_name = df['ct_dir_name']
mri_name = df['XNATSessionID']
id_dict = dict(zip(ct_name,mri_name))
ct_order = sorted(ct_name)
print(id_dict)
for count, name in enumerate(ct_order):
    print(f'======{count+1}=======')
    print('ct_name: ',name)
    print('mri_name: ',id_dict[name])

    print('======================')

In [ ]:
sor_dir = r'S:\StrokeAI\Stroke_Dataset'
origin_ctdir =origin_ctdir = r'S:\StrokeAI\Stroke_Dataset\CT'
origin_mridir = r'S:\StrokeAI\Stroke_Dataset\MRI'
ct_dir = r'S:\StrokeAI\Stroke_Dataset\Stroke_CT_Selected'
mri_dir = r'S:\StrokeAI\Stroke_Dataset\Stroke_Label_Selected'
not_processed = []
for count, name in enumerate(ct_order):
    print('============{}==========='.format(count+1))
    try:
    #if count== 2:
       # break
        ct_file = sitk.ReadImage(os.path.join(origin_ctdir,name,f'{name}_skullstrip.nii.gz'))
        label_file = sitk.ReadImage(os.path.join(origin_mridir,name,f'{name}_label.nii.gz'))
        print(os.path.join(origin_ctdir,name,f'{name}_skullstrip.nii.gz'))
        print(os.path.join(origin_mridir,name,f'{name}_label.nii.gz'))
        os.chdir(ct_dir)
        sitk.WriteImage(ct_file,'Stroke_{}.nii.gz'.format(count+1))
        print('CT Transfered -> Stroke_{}.nii.gz'.format(count+1))
        os.chdir(mri_dir)
        sitk.WriteImage(label_file,'Stroke_{}.nii.gz'.format(count+1))
        print('MRI Transfered -> Stroke_{}.nii.gz'.format(count+1))

    except :
        not_processed.append(name)
        print("Not_Processed:", name)
        continue

In [ ]:
not_processed

In [ ]:
case_diss = """
129

"""

In [ ]:
not_processed = []
CT_list = []
Label_list = []
import pandas as pd
for count, name in enumerate(ct_order):
    print('============{}==========='.format(count+1))
    try:
    #if count== 2:
       # break
        CT_list.append(name)
        Label_list.append('Stroke_{}'.format(count+1))
        
#         ct_file = sitk.ReadImage(os.path.join(origin_ctdir,name,f'{name}_skullstrip.nii.gz'))
#         label_file = sitk.ReadImage(os.path.join(origin_mridir,name,f'{name}_label.nii.gz'))
#         print(os.path.join(origin_ctdir,name,f'{name}_skullstrip.nii.gz'))
#         print(os.path.join(origin_mridir,name,f'{name}_label.nii.gz'))
#         os.chdir(ct_dir)
#         sitk.WriteImage(ct_file,'Stroke_{}.nii.gz'.format(count+1))
        print('CT Origin -> {}.nii.gz'.format(name))
        print('CT Transfered -> Stroke_{}.nii.gz'.format(count+1))
#         os.chdir(mri_dir)
#         sitk.WriteImage(label_file,'Stroke_{}.nii.gz'.format(count+1))
#         print('MRI Transfered -> Stroke_{}.nii.gz'.format(count+1))

    except :
        not_processed.append(name)
        print("Not_Processed:", name)
        continue

In [ ]:
CTdict = dict(zip(CT_list,Label_list))
CTds = pd.DataFrame()
CTds['CT_Origin'] = CTdict.keys()
CTds['CT_transfer'] = CTdict.values()

In [ ]:
CTds.iloc[247]

In [ ]:
import os
os.chdir(r'S:\StrokeAI')
CTds.to_csv('Linking.csv')

In [ ]:
selx = sitk.ElastixImageFilter()